In [198]:
import pandas as pd
import numpy as np
import os
import re
import seaborn as sb
import matplotlib
import matplotlib.pyplot as plt
import scipy.stats as st
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.decomposition import PCA 
from sklearn.preprocessing import LabelEncoder, PolynomialFeatures
from sklearn import metrics, preprocessing
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier


In [73]:
train = pd.read_csv(r'C:\Users\wonca\Desktop\DA_Proj\TitanicData\train.csv',header = 'infer',encoding = 'latin1')
test = pd.read_csv(r'C:\Users\wonca\Desktop\DA_Proj\TitanicData\test.csv',header = 'infer',encoding = 'latin1')

In [74]:
df = pd.concat([train,test],ignore_index=True,sort=True)

In [75]:
title = []
titleReg = re.compile(r',\s+(\w+).')
for name in df.Name : 
    title.append(titleReg.search(name).group(1))
title = pd.Series(title,name = 'title')
title[(title=='Dr') | (title == 'Rev' )| (title == 'Major') | (title == 'Col') | (title == 'Capt') | (title == 'Don') |( title == 'Sir') | (title == 'Jonkheer')] = 'Mr'
title[title == 'Mlle'] = 'Miss'
title[(title == 'Lady')|(title == 'Mme')|(title == 'Ms')|(title == 'the')|(title=='Dona')] = 'Mrs'
title.value_counts()

Mr        783
Miss      262
Mrs       203
Master     61
Name: title, dtype: int64

In [76]:
df = df.join(title)

In [77]:
meanAge = df.loc[:,['Age','title']].groupby('title').mean()
n = df.shape[0]
Age = []
for i in range(n):
    if np.isnan(df.Age[i]):
        if df.title[i] == 'Mr': Age.append(float(meanAge.loc['Mr']))
        elif df.title[i] == 'Mrs' : Age.append(float(meanAge.loc['Mrs']))
        elif df.title[i] == 'Master' : Age.append(float(meanAge.loc['Master']))
        else : Age.append(float(meanAge.loc['Miss']))
    else:
        Age.append(df.Age[i])
df.Age = pd.Series(Age,name='Age')

In [78]:
df.loc[61,'Embarked'] = 'C'
df.loc[829,'Embarked'] = 'C'

In [79]:
df.drop(columns='Cabin',inplace=True)
df.drop(columns= 'Name',inplace = True)
df.drop(columns = 'PassengerId',inplace = True)
df['FamSize'] = df['SibSp']+df['Parch']+1
df.drop(columns='SibSp',inplace = True)
df.drop(columns= 'Parch',inplace = True)
df.drop(columns = 'Ticket',inplace = True)

In [80]:
n = df.shape[0]
fam = []
for i in range(n):
    if df.FamSize[i] == 1:fam.append('single')
    elif 1<df.FamSize[i]<5 : fam.append('small')
    elif 5<df.FamSize[i]<8 : fam.append('big')
    else : fam.append('large')
df['Fam'] = pd.Series(fam)

In [81]:
#Age
LE = LabelEncoder()
df['AgeCut'] = LE.fit_transform(pd.qcut(df.Age,5))
ageLi = list(df.loc[:,['Survived','AgeCut']].groupby('AgeCut').mean().sort_values(by = 'Survived',ascending = True).index)
ageDic = {ageLi[i]:i for i in range(0,len(ageLi))}
df['AgeCut'] = df.AgeCut.apply(lambda x : ageDic[x])

pd.qcut(df.Fare,20).unique()

In [122]:
#Fare
LE = LabelEncoder()
df['FareCut'] = LE.fit_transform(pd.qcut(df.Fare,20))

fl = list(df.loc[:,['FareCut','Survived']].groupby('FareCut').mean().sort_values(by = 'Survived').index)
fdic = {fl[i]: i for i in range(0,len(fl))}
df['FareCut'] = df.FareCut.apply(lambda x : fdic[x])

In [123]:
# FAM 은 더미변수로 치환
fd = pd.get_dummies(df.Fam,drop_first= True, prefix= 'Fam')
#df = df.join(fd)

In [124]:
# Sex
LE = LabelEncoder()
df.Sex = LE.fit_transform(df.Sex)

In [125]:
# Embarked
dic = {'S':0,'Q':1,'C':2}
df.Embarked = df.Embarked.apply(lambda x : dic[x])

In [126]:
# Titie
td = pd.get_dummies(df.title, drop_first= True, prefix= 'title')
df= df.join(td)

In [127]:
dfFin = df.drop(columns=['title','Fam','Age','Fare'])

In [128]:
dfFin.head()

,Embarked,Pclass,Sex,Survived,FamSize,AgeCut,FareCut,title_Miss,title_Mr,title_Mrs
0,0,3,1,0.0,2,3,3,0,1,0
1,2,1,0,1.0,2,4,15,0,0,1
2,0,3,0,1.0,1,3,4,1,0,0
3,0,1,0,1.0,2,4,16,0,0,1
4,0,3,1,0.0,1,4,4,0,1,0


In [144]:
trainDat = dfFin.loc[:890,:]
testDat = dfFin.loc[891:,:].drop(columns = 'Survived')

In [145]:
xTrain = trainDat.drop(columns='Survived')
yTrain = trainDat.Survived

In [159]:
glm = LogisticRegression()
glm.fit(xTrain,yTrain)
yPred = glm.predict(testDat)

d:\pl\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [179]:
yPred = pd.Series(yPred,name = 'Survived')

In [187]:
yPred = yPred.astype(int)

In [188]:
sub = pd.concat([test.PassengerId, yPred], axis = 1)

In [189]:
sub.to_csv('sub3.csv',index = False,encoding='latin1')

In [194]:
C_grid = [0.1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
gamma_grid = [0.0001, 0.0002,0.0005, 0.0006, 0.0007, 0.0008, 0.001,  0.002, 0.005]
parameters = {'C': C_grid, 'gamma' : gamma_grid}
gridCV = GridSearchCV(SVC(kernel='rbf'), parameters, cv=5);
gridCV.fit(xTrain, yTrain)
best_C = gridCV.best_params_['C']
best_gamma = gridCV.best_params_['gamma']
SVM_best = SVC(C=best_C,gamma=best_gamma)
SVM_best.fit(xTrain, yTrain);
y_pred = SVM_best.predict(testDat)

In [195]:
yPred = yPred.astype(int)
sub = pd.concat([test.PassengerId, yPred], axis = 1)
sub.to_csv('sub4.csv',index = False,encoding='latin1')

## randomForest

In [196]:
# Tree 하고는 스텝에서 조금 차이가 있음
estimator_grid = np.arange(1, 30, 5)
depth_grid = np.arange(1, 10, 2)
parameters = {'n_estimators': estimator_grid, 'max_depth': depth_grid}
gridCV = GridSearchCV(RandomForestClassifier(), param_grid=parameters, cv=10)
gridCV.fit(xTrain, yTrain)
best_n_estim = gridCV.best_params_['n_estimators']
best_depth = gridCV.best_params_['max_depth']
RF_best = RandomForestClassifier(max_depth=best_depth,n_estimators=best_n_estim,random_state=3)
RF_best.fit(xTrain, yTrain);
YpredRF = RF_best.predict(testDat)

In [199]:
# estimator_grid
estimator_grid = np.arange(30, 80, 10)
learning_rate_grid = np.array([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])
parameters = {'n_estimators': estimator_grid, 'learning_rate': learning_rate_grid}
gridCV = GridSearchCV(AdaBoostClassifier(), param_grid=parameters, cv=10)
gridCV.fit(xTrain, yTrain)
best_n_estim = gridCV.best_params_['n_estimators']
best_learn_rate = gridCV.best_params_['learning_rate']
AB_best = AdaBoostClassifier(n_estimators=best_n_estim,learning_rate=best_learn_rate,random_state=3)
AB_best.fit(xTrain, yTrain);
YpredAB = AB_best.predict(testDat)


In [202]:
YpredRF = pd.Series(YpredRF,name = 'Survived')
YpredRF = YpredRF.astype(int)
sub = pd.concat([test.PassengerId, YpredRF], axis = 1)
sub.to_csv('subRF.csv',index = False,encoding='latin1')

YpredAB = pd.Series(YpredAB,name = 'Survived')
YpredAB = YpredRF.astype(int)
sub = pd.concat([test.PassengerId, YpredAB], axis = 1)
sub.to_csv('subAB.csv',index = False,encoding='latin1')




In [201]:
YpredRF

array([0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1,
       0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,